In [51]:
import pandas as pd

df = pd.read_csv('cleaned_diamond_data.csv').rename(columns = {'Unnamed: 0' : 'id'})
df.drop('id',inplace=True,axis=1)
df.head(5)

,carat,clarity,color,cut,x,y,z,depth,table,price
0,0.5,IF,D,IDEAL,5.1,5.15,3.20,61.5,56.0,3000.0
1,0.7,VVS2,E,PREMIUM,5.7,5.49,3.52,62.0,59.0,4500.0
2,0.5,SI2,H,GOOD,4.3,4.31,3.90,62.3,56.0,700.0
3,1.2,IF,D,IDEAL,5.9,6.82,4.20,61.7,58.0,10000.0
4,0.9,I1,J,FAIR,6.0,5.49,3.70,61.7,56.0,2400.0


In [52]:
df.shape

(195, 10)

In [50]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# L

# Define the features and target
X = df.drop('price', axis=1)
y = df['price']

# Add polynomial features
degree = 2  # You can change the degree of the polynomial
X_poly = pd.DataFrame(np.column_stack([X ** i for i in range(1, degree + 1)]), columns=[f'x{i}' for i in range(1, (degree * X.shape[1]) + 1)])

# Add constant for intercept
X_poly = sm.add_constant(X_poly)

# Fit the model
model = sm.OLS(y, X_poly).fit()

# Perform backward elimination
while (model.pvalues.drop('const') > 0.05).any():
    max_p_value = model.pvalues.drop('const').max()
    if max_p_value > 0.05:
        remove_feature = model.pvalues.drop('const').idxmax()
        X_poly.drop(remove_feature, axis=1, inplace=True)
        model = sm.OLS(y, X_poly).fit()

# Display the summary
print(model.summary())


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Przekształć dane obiektowe na liczby
le = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

df['clarity'] = le.fit_transform(df['clarity'])
df['cut'] = le2.fit_transform(df['cut'])
df['color'] = le3.fit_transform(df['color'])

In [53]:
import statsmodels.formula.api as smf

model = smf.ols(formula="price ~ carat+ C(clarity) + C(color) +  C(cut) + depth+ table + x+ y+z ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     61.03
Date:                Wed, 31 Jan 2024   Prob (F-statistic):           1.92e-69
Time:                        12:07:17   Log-Likelihood:                -1632.5
No. Observations:                 195   AIC:                             3311.
Df Residuals:                     172   BIC:                             3386.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -6.195e+04   2.01e+04     -3.083      0.002   -1.02e+05   -2.23e+04
C(clarity)[T.IF]     1977.1647    346.640      5.704      0.000    1292.948    2661.382
C(clarity)[T.SI1]    1200.2787    311.353      3.855      0.000     585.715    1814.843
C(clarity)[T.SI2]    1051.2223    287.357      3.658      0.000     484.022    1618.423
C(clarity)[T.VVS1]   1782.3000    308.001      5.787      0.000    1174.351    2390.249
C(clarity)[T.VVS2]   1935.1006    302.749      6.392      0.000    1337.518    2532.683
C(color)[T.D]         782.5616    515.794      1.517      0.131    -235.540    1800.663
C(color)[T.E]         323.3708    489.738      0.660      0.510    -643.299    1290.041
C(color)[T.F]         551.9521    469.163      1.176      0.241    -374.107    1478.011
C(color)[T.G]         698.0289    458.601      1.522      0.130    -207.182    1603.240
C(color)[T.H]         400.5006    480.608      0.833      0.406    -548.148    1349.149
C(color)[T.I]         732.7104    471.126      1.555      0.122    -197.224    1662.644
C(color)[T.J]          39.4396    579.094      0.068      0.946   -1103.607    1182.486
C(cut)[T.GOOD]        206.1474    268.123      0.769      0.443    -323.087     735.382
C(cut)[T.IDEAL]       396.9276    301.862      1.315      0.190    -198.903     992.759
C(cut)[T.PREMIUM]     718.5394    296.882      2.420      0.017     132.539    1304.540
C(cut)[T.VERY GOOD]   336.8313    285.035      1.182      0.239    -225.785     899.448
carat                2271.8907    445.694      5.097      0.000    1392.157    3151.624
depth                 678.5779    316.577      2.143      0.033      53.703    1303.453
table                  99.2095     69.640      1.425      0.156     -38.249     236.669
x                    2848.9456    287.959      9.894      0.000    2280.556    3417.335
y                    -355.3671    206.509     -1.721      0.087    -762.986      52.252
z                     203.1252    336.508      0.604      0.547    -461.091     867.341
==============================================================================
Omnibus:                       75.207   Durbin-Watson:                   2.070
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              459.359
Skew:                          -1.312   Prob(JB):                    1.78e-100
Kurtosis:                      10.046   Cond. No.                     2.13e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [55]:
model = smf.ols(formula="price ~ carat+ C(clarity) + C(color) +  C(cut) + depth+ table + x+ y ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     64.15
Date:                Wed, 31 Jan 2024   Prob (F-statistic):           2.83e-70
Time:                        12:17:35   Log-Likelihood:                -1632.7
No. Observations:                 195   AIC:                             3309.
Df Residuals:                     173   BIC:                             3381.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -6.217e+04   2.01e+04     -3.100      0.002   -1.02e+05   -2.26e+04
C(clarity)[T.IF]     1981.8695    345.915      5.729      0.000    1299.111    2664.628
C(clarity)[T.SI1]    1210.7876    310.294      3.902      0.000     598.338    1823.237
C(clarity)[T.SI2]    1068.9084    285.334      3.746      0.000     505.725    1632.092
C(clarity)[T.VVS1]   1805.4579    305.041      5.919      0.000    1203.377    2407.538
C(clarity)[T.VVS2]   1937.1221    302.174      6.411      0.000    1340.699    2533.545
C(color)[T.D]         814.6887    512.097      1.591      0.113    -196.074    1825.452
C(color)[T.E]         357.6319    485.543      0.737      0.462    -600.719    1315.983
C(color)[T.F]         589.1420    464.245      1.269      0.206    -327.172    1505.456
C(color)[T.G]         730.7025    454.558      1.608      0.110    -166.491    1627.896
C(color)[T.H]         431.0598    477.055      0.904      0.367    -510.538    1372.658
C(color)[T.I]         784.6713    462.344      1.697      0.091    -127.890    1697.232
C(color)[T.J]          37.1315    578.017      0.064      0.949   -1103.741    1178.004
C(cut)[T.GOOD]        219.2701    266.749      0.822      0.412    -307.231     745.771
C(cut)[T.IDEAL]       402.2209    301.180      1.335      0.183    -192.239     996.681
C(cut)[T.PREMIUM]     743.9613    293.339      2.536      0.012     164.977    1322.946
C(cut)[T.VERY GOOD]   333.7234    284.464      1.173      0.242    -227.744     895.191
carat                2285.0635    444.341      5.143      0.000    1408.037    3162.090
depth                 688.3023    315.585      2.181      0.031      65.410    1311.195
table                  97.0254     69.418      1.398      0.164     -39.990     234.041
x                    2940.2844    244.547     12.023      0.000    2457.604    3422.965
y                    -373.2178    204.005     -1.829      0.069    -775.877      29.442
==============================================================================
Omnibus:                       73.583   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              446.073
Skew:                          -1.280   Prob(JB):                     1.37e-97
Kurtosis:                       9.953   Cond. No.                     2.13e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
model = smf.ols(formula="price ~ carat+ C(clarity) + C(color) +  C(cut) + depth + x+ y ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     66.90
Date:                Wed, 31 Jan 2024   Prob (F-statistic):           8.90e-71
Time:                        12:17:51   Log-Likelihood:                -1633.8
No. Observations:                 195   AIC:                             3310.
Df Residuals:                     174   BIC:                             3378.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -5.492e+04   1.94e+04     -2.828      0.005   -9.32e+04   -1.66e+04
C(clarity)[T.IF]     2055.1287    342.857      5.994      0.000    1378.435    2731.823
C(clarity)[T.SI1]    1266.5656    308.559      4.105      0.000     657.565    1875.566
C(clarity)[T.SI2]    1099.9391    285.247      3.856      0.000     536.949    1662.929
C(clarity)[T.VVS1]   1849.8846    304.210      6.081      0.000    1249.467    2450.302
C(clarity)[T.VVS2]   1997.5566    299.883      6.661      0.000    1405.680    2589.433
C(color)[T.D]         815.9081    513.498      1.589      0.114    -197.578    1829.394
C(color)[T.E]         325.9319    486.340      0.670      0.504    -633.954    1285.818
C(color)[T.F]         619.2134    465.015      1.332      0.185    -298.583    1537.010
C(color)[T.G]         752.4723    455.534      1.652      0.100    -146.612    1651.556
C(color)[T.H]         459.3972    477.928      0.961      0.338    -483.886    1402.680
C(color)[T.I]         750.6644    462.967      1.621      0.107    -163.089    1664.418
C(color)[T.J]         -19.1964    578.188     -0.033      0.974   -1160.361    1121.968
C(cut)[T.GOOD]        298.8751    261.310      1.144      0.254    -216.871     814.621
C(cut)[T.IDEAL]       472.5893    297.754      1.587      0.114    -115.086    1060.265
C(cut)[T.PREMIUM]     876.3378    278.388      3.148      0.002     326.885    1425.791
C(cut)[T.VERY GOOD]   409.7040    279.986      1.463      0.145    -142.902     962.310
carat                2318.2806    444.919      5.211      0.000    1440.148    3196.413
depth                 656.2367    315.611      2.079      0.039      33.317    1279.156
x                    2980.2566    243.534     12.238      0.000    2499.596    3460.917
y                    -395.4247    203.942     -1.939      0.054    -797.943       7.094
==============================================================================
Omnibus:                       74.963   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              475.665
Skew:                          -1.292   Prob(JB):                    5.14e-104
Kurtosis:                      10.202   Cond. No.                     1.53e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
model = smf.ols(formula="price ~ carat+ C(clarity) + C(color) +  C(cut) + table + x+ y ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     2.103
Date:                Wed, 31 Jan 2024   Prob (F-statistic):            0.00557
Time:                        11:31:46   Log-Likelihood:                -2379.9
No. Observations:                 200   AIC:                             4802.
Df Residuals:                     179   BIC:                             4871.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -2.181e+05    1.3e+05     -1.673      0.096   -4.75e+05    3.91e+04
C(clarity)[T.1] -8942.7706   1.12e+04     -0.800      0.425    -3.1e+04    1.31e+04
C(clarity)[T.2] -1.369e+04   1.03e+04     -1.333      0.184    -3.4e+04    6579.367
C(clarity)[T.3]  -351.0370   9473.157     -0.037      0.970    -1.9e+04    1.83e+04
C(clarity)[T.4] -9039.9630   1.01e+04     -0.892      0.374    -2.9e+04     1.1e+04
C(clarity)[T.5] -2950.0451      1e+04     -0.294      0.769   -2.28e+04    1.69e+04
C(color)[T.1]    3.557e+04    1.7e+04      2.091      0.038    2006.207    6.91e+04
C(color)[T.2]    -443.9582   1.64e+04     -0.027      0.978   -3.28e+04    3.19e+04
C(color)[T.3]    6306.6428   1.57e+04      0.402      0.688   -2.46e+04    3.73e+04
C(color)[T.4]    1.589e+04   1.53e+04      1.038      0.301   -1.43e+04    4.61e+04
C(color)[T.5]    5099.6663    1.6e+04      0.319      0.750   -2.65e+04    3.67e+04
C(color)[T.6]    1.124e+04   1.55e+04      0.725      0.469   -1.93e+04    4.18e+04
C(color)[T.7]    4704.5401   1.96e+04      0.241      0.810   -3.39e+04    4.33e+04
C(cut)[T.1]     -1.625e+04   8802.715     -1.846      0.066   -3.36e+04    1117.065
C(cut)[T.2]     -9940.1104   9994.649     -0.995      0.321   -2.97e+04    9782.386
C(cut)[T.3]     -7850.6845   9673.622     -0.812      0.418   -2.69e+04    1.12e+04
C(cut)[T.4]      3395.9052   9434.899      0.360      0.719   -1.52e+04     2.2e+04
carat           -1.168e+04   1.47e+04     -0.792      0.429   -4.08e+04    1.74e+04
table            4007.2903   2308.755      1.736      0.084    -548.588    8563.168
x                2.494e+04   7862.290      3.173      0.002    9428.983    4.05e+04
y               -2.369e+04   6605.190     -3.586      0.000   -3.67e+04   -1.07e+04
==============================================================================
Omnibus:                      232.761   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7462.412
Skew:                           4.854   Prob(JB):                         0.00
Kurtosis:                      31.306   Cond. No.                     2.81e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.81e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:

model = smf.ols(formula="price ~ carat+ clarity + table + x+ y ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     4.154
Date:                Wed, 31 Jan 2024   Prob (F-statistic):            0.00131
Time:                        11:13:54   Log-Likelihood:                -2390.9
No. Observations:                 200   AIC:                             4794.
Df Residuals:                     194   BIC:                             4814.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.077e+05    1.2e+05     -1.735      0.084   -4.44e+05    2.84e+04
carat      -1.264e+04   1.43e+04     -0.887      0.376   -4.08e+04    1.55e+04
clarity    -1373.2232   1623.646     -0.846      0.399   -4575.487    1829.041
table       3549.2413   2105.571      1.686      0.093    -603.509    7701.992
x           2.477e+04   7511.023      3.298      0.001    9960.665    3.96e+04
y           -2.02e+04   6479.293     -3.117      0.002    -3.3e+04   -7416.486
==============================================================================
Omnibus:                      266.358   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13184.970
Skew:                           5.895   Prob(JB):                         0.00
Kurtosis:                      40.990   Cond. No.                     2.55e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.55e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:

model = smf.ols(formula="price ~ carat+  table + x+ y ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     5.021
Date:                Wed, 31 Jan 2024   Prob (F-statistic):           0.000714
Time:                        11:14:12   Log-Likelihood:                -2391.2
No. Observations:                 200   AIC:                             4792.
Df Residuals:                     195   BIC:                             4809.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.991e+05   1.19e+05     -1.670      0.096   -4.34e+05     3.6e+04
carat      -1.199e+04   1.42e+04     -0.843      0.400   -4.01e+04    1.61e+04
table       3358.2746   2091.901      1.605      0.110    -767.382    7483.931
x            2.43e+04   7484.594      3.247      0.001    9539.022    3.91e+04
y          -2.005e+04   6472.193     -3.097      0.002   -3.28e+04   -7282.670
==============================================================================
Omnibus:                      266.865   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13186.884
Skew:                           5.917   Prob(JB):                         0.00
Kurtosis:                      40.979   Cond. No.                     2.54e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.54e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:

model = smf.ols(formula="price ~  x+ y ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     8.408
Date:                Wed, 31 Jan 2024   Prob (F-statistic):           0.000313
Time:                        11:15:11   Log-Likelihood:                -2392.9
No. Observations:                 200   AIC:                             4792.
Df Residuals:                     197   BIC:                             4802.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2164.2458   2.44e+04     -0.089      0.929   -5.02e+04    4.59e+04
x           2.231e+04   5445.715      4.096      0.000    1.16e+04     3.3e+04
y          -2.061e+04   6386.758     -3.227      0.001   -3.32e+04   -8014.028
==============================================================================
Omnibus:                      271.021   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14050.509
Skew:                           6.059   Prob(JB):                         0.00
Kurtosis:                      42.233   Cond. No.                         75.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
df.corr()

,carat,clarity,color,cut,x,y,z,depth,table,price
carat,1.000000,0.038356,-0.083564,0.006939,0.780244,0.536506,0.632880,0.266465,0.223510,0.121116
clarity,0.038356,1.000000,-0.114816,-0.117444,0.084047,0.041925,0.131219,0.110232,0.124603,-0.020934
color,-0.083564,-0.114816,1.000000,-0.130213,-0.160668,-0.074220,-0.106440,0.071364,-0.195118,-0.100818
cut,0.006939,-0.117444,-0.130213,1.000000,0.003132,-0.045412,-0.003561,-0.025157,0.242646,0.073736
x,0.780244,0.084047,-0.160668,0.003132,1.000000,0.758170,0.779332,0.314368,0.238023,0.173042
y,0.536506,0.041925,-0.074220,-0.045412,0.758170,1.000000,0.516829,0.207827,0.101011,-0.012701
z,0.632880,0.131219,-0.106440,-0.003561,0.779332,0.516829,1.000000,0.314184,0.188741,0.137292
depth,0.266465,0.110232,0.071364,-0.025157,0.314368,0.207827,0.314184,1.000000,0.027876,0.050543
table,0.223510,0.124603,-0.195118,0.242646,0.238023,0.101011,0.188741,0.027876,1.000000,0.171113
price,0.121116,-0.020934,-0.100818,0.073736,0.173042,-0.012701,0.137292,0.050543,0.171113,1.000000


In [ ]:
import statsmodels.formula.api as smf

model = smf.ols(formula="price ~ carat ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     2.948
Date:                Tue, 30 Jan 2024   Prob (F-statistic):             0.0876
Time:                        16:34:20   Log-Likelihood:                -2399.6
No. Observations:                 200   AIC:                             4803.
Df Residuals:                     198   BIC:                             4810.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -96.3184   7166.174     -0.013      0.989   -1.42e+04     1.4e+04
carat       1.564e+04   9112.356      1.717      0.088   -2324.826    3.36e+04
==============================================================================
Omnibus:                      291.398   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19630.243
Skew:                           6.776   Prob(JB):                         0.00
Kurtosis:                      49.604   Cond. No.                         5.08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash import dash_table
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import plotly.graph_objects as go


# Wczytanie danych
df = pd.read_csv('cleaned_diamond_data.csv').rename(columns={'Unnamed: 0': 'id'})
df.drop('id', inplace=True, axis=1)

# Label Encoding dla danych kategorycznych
le = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

df['clarity'] = le.fit_transform(df['clarity'])
df['cut'] = le2.fit_transform(df['cut'])
df['color'] = le3.fit_transform(df['color'])

# Podział na zbiór treningowy i testowy
y = df['price']
X = df.drop(columns=['price'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Budowa modelu regresji
model = LinearRegression()

# Eliminacja wsteczna - RFE (Recursive Feature Elimination)
selector = RFE(model, step=1)
selector = selector.fit(X_train, y_train)

# Wybrane istotne zmienne
selected_features = X_train.columns[selector.support_]
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Dopasowanie modelu na wybranych zmiennych
model.fit(X_train_selected, y_train)

#odkecenie labeli
df['clarity'] = le.inverse_transform(df['clarity'])
df['cut'] = le2.inverse_transform(df['cut'])
df['color'] = le3.inverse_transform(df['color'])


# Inicjalizacja aplikacji Dash
app = dash.Dash(__name__)

# Opcje dla kontrolki wyboru kategorii
category_options = ['Rozkład zmiennych', 'Zależność ceny od innych zmiennych', 'Liczebność kategorii', 'Wizualizacja modelu regresji']

# Opcje dla kontrolki wyboru zmiennej
variable_options = ['carat', 'clarity', 'color', 'cut', 'x', 'y', 'z', 'depth', 'table']

# Układ strony
app.layout = html.Div(children=[
    html.H1(children='Diamond Data Dashboard'),

    # Kontrolka wyboru kategorii
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in category_options],
        value='Rozkład zmiennych',
        multi=False,
        style={'width': '50%'},
        clearable=False
    ),

    # Kontrolka wyboru zmiennej
    dcc.Dropdown(
        id='variable-dropdown',
        options=[{'label': variable, 'value': variable} for variable in variable_options],
        value='carat',
        multi=False,
        style={'width': '50%'},
        clearable=False
    ),

    # Wykres
    dcc.Graph(id='visualization-plot'),

    # Tabela z próbką danych
    html.Div([
        html.H3('Próbka danych'),
        dash_table.DataTable(
            id='sample-data-table',
            columns=[{'name': col, 'id': col} for col in df.columns],
            data=df.sample(10).to_dict('records')
        )
    ])
])


# Funkcja do aktualizacji wykresu i tabeli na podstawie wybranej kategorii i zmiennej
@app.callback(
    [Output('visualization-plot', 'figure'),
     Output('sample-data-table', 'data')],
    [Input('category-dropdown', 'value'),
     Input('variable-dropdown', 'value')]
)
def update_plots(selected_category, selected_variable):
    if selected_category == 'Rozkład zmiennych':
        # Aktualizacja wykresu - Rozkład zmiennej
        fig = px.histogram(df, x=selected_variable, nbins=50, title=f'Rozkład zmiennej "{selected_variable}"')
    elif selected_category == 'Zależność ceny od innych zmiennych':
        # Aktualizacja wykresu - Zależność ceny od innych zmiennych
        if selected_variable == 'carat':
            fig = px.scatter(x=X_test_selected[selected_variable], y=y_test / X_test_selected[selected_variable],
                             labels={'x': selected_variable, 'y': 'Cena / Carat'},
                             title=f'Zależność ceny od zmiennej "{selected_variable}"',
                             mode='markers', template='plotly_white')
        else:
            y_pred = model.predict(X_test_selected)
            fig = px.scatter(x=y_test, y=y_pred, labels={'x': 'Rzeczywista cena', 'y': 'Przewidziana cena'},
                             title='Zależność ceny od innych zmiennych', mode='markers', template='plotly_white')
            fig.add_trace(px.scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], mode='lines',
                                     name='Idealna zależność', template='plotly_white').data[0])
    elif selected_category == 'Liczebność kategorii':
        # Aktualizacja wykresu - Liczebność kategorii
        fig = px.bar(df[selected_variable].value_counts(), x=df[selected_variable].unique(),
                     title=f'Liczebność kategorii "{selected_variable}"', labels={'x': selected_variable, 'y': 'Liczba'},
                     template='plotly_white')
    elif selected_category == 'Wizualizacja modelu regresji':
    # Aktualizacja wykresu - Wizualizacja modelu regresji
            y_pred = model.predict(X_test_selected)
            fig = px.scatter(x=y_test, y=y_pred, labels={'x': 'Rzeczywista cena', 'y': 'Przewidziana cena'},
                     title='Wizualizacja modelu regresji', mode='markers', template='plotly_white')

    # Dodanie linii idealnej zależności
    ideal_line = go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)],
                            mode='lines', name='Idealna zależność', line={'dash': 'dash'})
    fig.add_trace(ideal_line)


    # Aktualizacja tabeli
    sample_data = df.sample(10).to_dict('records')

    return fig, sample_data


# Uruchomienie aplikacji
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: scatter() got an unexpected keyword argument 'mode'

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: scatter() got an unexpected keyword argument 'mode'

